In [1]:
import pandas as pd
from tqdm import tqdm

## Load Alarms with Lavori Programmati

In [2]:
mob = pd.read_parquet("alarms datasets/mob/[new]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [5]:
adsl = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")
pdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet")
ptn = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")
sdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")

In [3]:
mob["link_type"] = "None"

In [4]:
mob = mob[["alarm_id", "std_probable_cause_no","link_type", "lp_id", "lp_type"]]

In [6]:

adsl = adsl[["alarm_id", "std_probable_cause_no","link_type", "lp_id", "lp_type"]]
pdh = pdh[["alarm_id", "std_probable_cause_no", "link_type","lp_id", "lp_type"]]
ptn = ptn[["alarm_id", "std_probable_cause_no","link_type", "lp_id", "lp_type"]]
sdh = sdh[["alarm_id", "std_probable_cause_no", "link_type","lp_id", "lp_type"]]

In [7]:
alarms_slogan_lavori_programmati = pd.concat([mob, adsl, pdh, ptn, sdh])

In [8]:
alarms_slogan_lavori_programmati["link_type"] = alarms_slogan_lavori_programmati["link_type"].fillna("None")

In [9]:
alarms_slogan_lavori_programmati.to_parquet("alarms datasets/alarms_slogan_lavori_programmati.parquet")

## Merge Clusters and Alarms with Lavori Programmati

In [10]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")

In [11]:
alarms_slogan_lavori_programmati = pd.read_parquet("alarms datasets/alarms_slogan_lavori_programmati.parquet")

In [12]:
clusters_with_lavori_programmati = pd.merge(clusters, alarms_slogan_lavori_programmati, on="alarm_id", how="left")

In [13]:
clusters_with_lavori_programmati = clusters_with_lavori_programmati[
    ["alarm_id", "cluster_id2", "std_probable_cause_no","link_type", "ne_type", "lp_type"]
]

In [14]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_15,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_15,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_24,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_24,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_24,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
8073960,EFM;6591f08739628d00013784db,202312312354_8,base station connectivity problem,None,vnbts,None
8073961,EFM;6591f08cadefac0001744ea1,202312312354_8,base station connectivity problem,None,vnbts,None
8073962,EFMTX;6591ee0801021b00010facb4,202312312340_12,fermo totale apparato,l12,adsl,None
8073963,EFMTX;6591f046d8beb800016deaef,202312312344_9,oamloc locv,lsp,ptn3,None


In [15]:
rows_with_lp_type_not_none = clusters_with_lavori_programmati[clusters_with_lavori_programmati["lp_type"].notnull()]

In [16]:
rows_with_lp_type_not_none
clustersid2_distinct  = rows_with_lp_type_not_none["cluster_id2"].unique()
len(clustersid2_distinct)

3714

In [17]:
clusters_with_lavori_programmati['std_probable_cause_no'] = clusters_with_lavori_programmati['std_probable_cause_no'].fillna('None')

In [18]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_15,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_15,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_24,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_24,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_24,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
8073960,EFM;6591f08739628d00013784db,202312312354_8,base station connectivity problem,None,vnbts,None
8073961,EFM;6591f08cadefac0001744ea1,202312312354_8,base station connectivity problem,None,vnbts,None
8073962,EFMTX;6591ee0801021b00010facb4,202312312340_12,fermo totale apparato,l12,adsl,None
8073963,EFMTX;6591f046d8beb800016deaef,202312312344_9,oamloc locv,lsp,ptn3,None


In [19]:
filtered_df = clusters_with_lavori_programmati[clusters_with_lavori_programmati['lp_type'].apply(lambda x: x != 'None')]
filtered_df

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_15,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_15,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_24,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_24,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_24,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
8073960,EFM;6591f08739628d00013784db,202312312354_8,base station connectivity problem,None,vnbts,None
8073961,EFM;6591f08cadefac0001744ea1,202312312354_8,base station connectivity problem,None,vnbts,None
8073962,EFMTX;6591ee0801021b00010facb4,202312312340_12,fermo totale apparato,l12,adsl,None
8073963,EFMTX;6591f046d8beb800016deaef,202312312344_9,oamloc locv,lsp,ptn3,None


In [20]:
clusters_with_lavori_programmati['lp_type'] = clusters_with_lavori_programmati['lp_type'].replace([None], ['None'])

In [21]:
clusters_with_lavori_programmati['lp_type'] = clusters_with_lavori_programmati['lp_type'].str.replace(r"\[|\]|\'", "", regex=True)

In [22]:
unique_lp_type_values = clusters_with_lavori_programmati['lp_type'].apply(lambda x: str(x)).unique()
unique_lp_type_values

array(['None', 'APPARATI', 'CAVO', 'DWDM/PTN', 'REIX/UPGRADE',
       'CAMBI RELEASE', 'CAVO DWDM/PTN', 'CAVO APPARATI'], dtype=object)

In [23]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_15,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_15,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_24,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_24,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_24,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
8073960,EFM;6591f08739628d00013784db,202312312354_8,base station connectivity problem,None,vnbts,None
8073961,EFM;6591f08cadefac0001744ea1,202312312354_8,base station connectivity problem,None,vnbts,None
8073962,EFMTX;6591ee0801021b00010facb4,202312312340_12,fermo totale apparato,l12,adsl,None
8073963,EFMTX;6591f046d8beb800016deaef,202312312344_9,oamloc locv,lsp,ptn3,None


In [56]:
# clusters_with_lavori_programmati['ne_type_slogan_link_type'] = clusters_with_lavori_programmati.apply(lambda row: str(row.ne_type) + '_' + str(row.std_probable_cause_no + '_' + str(row.link_type)), axis=1)
# clusters_with_lavori_programmati['ne_type_slogan_link_type'] = clusters_with_lavori_programmati.apply(lambda row: str(row.ne_type) + '_' + str(row.std_probable_cause_no), axis=1)
clusters_with_lavori_programmati['ne_type_slogan_link_type'] = clusters_with_lavori_programmati.apply(lambda row: str(row.std_probable_cause_no), axis=1)

In [57]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type,ne_type_slogan_link_type
0,COS1P;136885076,202301010000_15,apparato isolato,None,ad16,None,apparato isolato
1,COS1P;136885115,202301010000_15,apparato isolato,None,ad16,None,apparato isolato
2,COS1P;136884642,202301010000_24,apparato isolato,None,adm4,None,apparato isolato
3,COS1P;136884882,202301010000_24,apparato isolato,None,ad16,None,apparato isolato
4,COS1P;136884883,202301010000_24,apparato isolato,None,ad16,None,apparato isolato
...,...,...,...,...,...,...,...
8073960,EFM;6591f08739628d00013784db,202312312354_8,base station connectivity problem,None,vnbts,None,base station connectivity problem
8073961,EFM;6591f08cadefac0001744ea1,202312312354_8,base station connectivity problem,None,vnbts,None,base station connectivity problem
8073962,EFMTX;6591ee0801021b00010facb4,202312312340_12,fermo totale apparato,l12,adsl,None,fermo totale apparato
8073963,EFMTX;6591f046d8beb800016deaef,202312312344_9,oamloc locv,lsp,ptn3,None,oamloc locv


## Create a Dataframe with Clusters and "GUASTO CAVO" label

In [58]:
clusters_grouped = clusters_with_lavori_programmati.groupby(["cluster_id2"])

### Labeling Rule

- GUASTO CAVO = 1 : if at least one alarm inside the cluster has a lp_type == 'CAVO'
- GUASTO CAVO = 0 : if there are no alarms with a lp_type == 'CAVO'
- No labeling (cluster not in the dataframe) : if whole number of alarms in the cluster have a lp_type == 'None' (no lavoro programmato info)

In [59]:
def check_cavo_alarm(group):
    if all(x == 'None' for x in group['lp_type'].values):
        return None
    return any(sublist == 'CAVO' for sublist in group['lp_type'].values if sublist is not None)

temp_df = clusters_grouped.apply(lambda group: pd.Series({
    'cluster_id2': group.name,
    'GUASTO CAVO': check_cavo_alarm(group)
}))

new_df = temp_df.dropna().reset_index(drop=True)

C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_7492\1928354519.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_df = clusters_grouped.apply(lambda group: pd.Series({


In [60]:
new_df.to_csv("real-time clusters/20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv", index=False)

## Create Cluster/Slogan Bitmap for lavoro programmato clusters

In [61]:
new_df = pd.read_csv("real-time clusters/20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv")

In [62]:
new_df = new_df[new_df['GUASTO CAVO'] == True]
new_df

,cluster_id2,GUASTO CAVO
8,202301100202_8,True
9,202301100204_2,True
11,202301100206_5,True
18,202301100220_1,True
19,202301100230_7,True
...,...,...
3709,202312200220_8,True
3710,202312200222_2,True
3711,202312200604_3,True
3712,202312210200_6,True


In [63]:
filtered_clusters = clusters_with_lavori_programmati[clusters_with_lavori_programmati['cluster_id2'].isin(new_df['cluster_id2'])]

In [64]:
filtered_clusters

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type,ne_type_slogan_link_type
78477,COS1P;137185782,202301100202_8,problemi wdm,None,olt,None,problemi wdm
78491,COS1P;137185837,202301100204_2,lossofwavelength och low,otr,olt,CAVO,lossofwavelength och low
78492,COS1P;137185975,202301100204_2,serversignalfailure och ssf,o40,olt,CAVO,serversignalfailure och ssf
78493,COS1P;137185862,202301100204_2,loss of signal los,None,olt,None,loss of signal los
78494,COS1P;137185864,202301100204_2,loss of signal los,None,olt,None,loss of signal los
...,...,...,...,...,...,...,...
7934668,EFMTX;6583901cf25c6e000142fd32,202312210208_7,ferf och bdi,odu0,pxm3,None,ferf och bdi
7934669,EFMTX;65838e02f25c6e000142fb7b,202312210200_6,serversignalfailure och ssf,odu2,olt,None,serversignalfailure och ssf
7934685,EFMTX;65838e03f25c6e000142fb7d,202312210200_6,serversignalfailure och ssf,odu2,olt,None,serversignalfailure och ssf
7934720,EFMTX;6583901bf25c6e000142fd2f,202312210208_7,ferf och bdi,odu2,pxm3,None,ferf och bdi


In [65]:
grouped_clusters = filtered_clusters.groupby("cluster_id2")
unique_slogans = clusters_with_lavori_programmati['ne_type_slogan_link_type'].unique()

In [66]:
len(unique_slogans)

116

In [67]:
# Definisci una funzione per calcolare la presenza di ciascuna tripla ne_type_slogan_link_type all'interno di un gruppo
def calculate_presence(group):
    presence = {slogan: 0 for slogan in unique_slogans}
    for slogan in group['ne_type_slogan_link_type']:
        presence[slogan] = 1
    return pd.Series(presence)

tqdm.pandas()

# Applica la funzione a ciascun gruppo e crea un nuovo dataframe con i risultati
result_df = grouped_clusters.progress_apply(calculate_presence)

  0%|          | 0/3047 [00:00<?, ?it/s]

100%|██████████| 3047/3047 [00:00<00:00, 4146.85it/s]


In [68]:
result_df.to_parquet("result_df_gt_2_lavoriprogrammati.parquet")

In [69]:
result_df

,apparato isolato,fermo parziale sito tec,None,alimentazione,signal degraded,loss of frame lof,guasto matrice,fermo parziale gsm,fermo totale gsm,fermo totale 5g,...,cardinitcard initializing,mpls tunnel rdi mpls tunnel rdi,vc unequipped ho vc unequipped,guasto controllore,vlan megaco ko,serversignalfailure odu ssf,unidentified specific problem non inviato da agent,replaceablemodulemissing eqpt,grave disservizio onuc,crdpwroff card power off xexc xexc
cluster_id2,,,,,,,,,,,,,,,,,,,,,
202301100202_8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100204_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100206_5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100220_1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301100230_7,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202312200220_8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202312200222_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202312200604_3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
guasto_cavo = pd.read_csv("real-time clusters/20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv")

In [71]:
guasto_cavo

,cluster_id2,GUASTO CAVO
0,202301090504_2,False
1,202301090504_3,False
2,202301090506_14,False
3,202301090506_4,False
4,202301090506_5,False
...,...,...
3709,202312200220_8,True
3710,202312200222_2,True
3711,202312200604_3,True
3712,202312210200_6,True


In [72]:
true_count = (guasto_cavo['GUASTO CAVO'] == True).sum()
false_count = (guasto_cavo['GUASTO CAVO'] == False).sum()
print("True count: ", true_count*100/(true_count+false_count))
print("False count: ", false_count*100/(true_count+false_count))

True count:  82.04092622509424
False count:  17.95907377490576
